In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("20181229_fut_ord.csv", low_memory=False, dtype={'SYMBOL':object,
                                                                  'SYSTEM':object,
                                                                  'TYPE':object,
                                                                  'MOMENT':object,
                                                                  'ID':object,
                                                                  'ACTION':object,
                                                                  'PRICE':object,
                                                                  'VOLUME':object,
                                                                  'ID_DEAL':object,
                                                                  'PRICE_DEAL':object
                                                                  })

In [3]:
df = df.drop(columns=['SYSTEM', 'ID_DEAL', 'PRICE_DEAL'])

In [4]:
df.to_csv('20181229_fut_ord_changed.csv', index=False)

In [5]:
tools = df.SYMBOL.unique()

In [15]:
import datetime
from datetime import timedelta

In [25]:
def create_ddl_partition_table(year, month, day):
    date_start = datetime.date(year=year, month=month, day=day)
    moment_start = str(date_start.year) + str(date_start.month) + str(date_start.day) + "000000000"
    delta = timedelta(days=1)
    date_end = date_start+delta
    moment_end = str(date_end.year) + str(date_end.month) + str(date_end.day) + "235959999"

    CREATE_PARTITION_DAY_TABLE = f"""CREATE TABLE current_rate_y{year}m{month}d{day}
                                PARTITION OF current_rate 
                                FOR VALUES FROM ({moment_start}) TO ({moment_end})
                                PARTITION BY LIST (tool);"""
    result_ddl_query = CREATE_PARTITION_DAY_TABLE + "\n"

    for tool in tools:
        # создание партиционированных таблиц по типу инструмента
        CREATE_PARTITION_TOOL_TABLE = f"""CREATE TABLE current_rate_y{year}m{month}d{day}_{tool} 
                                    PARTITION OF current_rate_y{year}m{month}d{day} 
                                    FOR VALUES IN ('{tool}');"""
        # создание индекса для поиска по id заявки
        CREATE_PARTITION_TOOL_TABLE_INDEX =f"""CREATE INDEX ON current_rate_y{year}m{month}d{day}_{tool} (id_request);"""
        result_ddl_query = result_ddl_query + CREATE_PARTITION_TOOL_TABLE + "\n" \
                         + CREATE_PARTITION_TOOL_TABLE_INDEX + "\n"

    # Open the file for writing
    with open(f"""sql_scripts/partition_table_y{year}m{month}d{day}.sql""", 'w') as f:
        f.write(result_ddl_query)
    

In [26]:
create_ddl_partition_table(2018, 12, 28)
create_ddl_partition_table(2018, 12, 29)
